In [4]:
# ==============================================================================
# BƯỚC 1: CÀI ĐẶT CÁC THƯ VIỆN CẦN THIẾT
# ==============================================================================
# Chạy dòng lệnh này trong terminal hoặc cell của Jupyter/Colab
!pip3 install networkx matplotlib scikit-learn python-louvain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 26.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.5 MB/s eta 0:00:00
  Created wheel for python-louvain: filename=python_louvain-0.16-py3-none-any.whl size=9460 sha256=5171685398ca15fca6d6f6a35051b8d50d9dd4a99973c270bd8f2ca425ea6c27
  Stored in directory: /Users/taminh/Library/Caches/pip/wheels/40/f1/e3/485b698c520fa0baee1d07897abc7b8d6479b7d199ce96f4af
Successfully built python-louvain

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [6]:


# ==============================================================================
# BƯỚC 2: IMPORT THƯ VIỆN
# ==============================================================================
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from community import community_louvain
from sklearn.cluster import SpectralClustering
from sklearn.metrics.cluster import adjusted_rand_score

# ==============================================================================
# BÀI TẬP 2: So sánh 3 thuật toán (Louvain, Girvan-Newman, Spectral)
# ==============================================================================
print("--- BÀI TẬP 2: So sánh các thuật toán ---")

# Sử dụng đồ thị kinh điển "Zachary's Karate Club"
G_karate = nx.karate_club_graph()

# 1. Thuật toán Louvain
partition_louvain = community_louvain.best_partition(G_karate)
num_louvain_communities = len(set(partition_louvain.values()))
print(f"[Louvain]        => Đã tìm thấy {num_louvain_communities} cộng đồng.")

# 2. Thuật toán Girvan-Newman
# Thuật toán này trả về một chuỗi các bước phân chia, ta lấy bước đầu tiên
comp_gn = nx.community.girvan_newman(G_karate)
partition_gn = tuple(sorted(c) for c in next(comp_gn))
num_gn_communities = len(partition_gn)
print(f"[Girvan-Newman]  => Đã tìm thấy {num_gn_communities} cộng đồng ở lần chia đầu tiên.")

# 3. Phân cụm theo Phổ (Spectral Clustering)
# Thuật toán này yêu cầu chúng ta phải chỉ định trước số cụm (k)
k = 2  # Giả sử ta muốn tìm 2 cụm (dựa theo câu chuyện gốc của đồ thị Karate)
adj_matrix = nx.to_numpy_array(G_karate)
sc = SpectralClustering(n_clusters=k, affinity='precomputed', assign_labels='kmeans', random_state=42)
partition_spectral = sc.fit_predict(adj_matrix)
print(f"[Spectral]       => Đã tìm thấy {k} cộng đồng như đã yêu cầu.")
print("-" * 50 + "\n")


# ==============================================================================
# BÀI TẬP 3: Mô phỏng cộng đồng biến đổi và theo dõi Modularity
# ==============================================================================
print("--- BÀI TẬP 3: Mô phỏng cộng đồng biến đổi ---")

# Tạo một đồ thị có cấu trúc cộng đồng rõ rệt (T=1)
sizes = [25, 25, 25]  # 3 cộng đồng, mỗi cộng đồng 25 nút
# Xác suất kết nối: cao bên trong (0.7), thấp bên ngoài (0.02)
probs = [[0.7, 0.02, 0.02], [0.02, 0.7, 0.02], [0.02, 0.02, 0.7]]
G1 = nx.stochastic_block_model(sizes, probs, seed=42)

# Xác định các cộng đồng gốc (ground truth)
ground_truth_communities = [set(range(0, 25)), set(range(25, 50)), set(range(50, 75))]

# Tính Modularity ở T=1
modularity_t1 = nx.community.modularity(G1, ground_truth_communities)
print(f"Modularity tại T=1 (cấu trúc rõ rệt): {modularity_t1:.4f}")

# "Làm mờ" ranh giới cộng đồng bằng cách thêm các cạnh "cầu nối" ngẫu nhiên (T=2)
G2 = G1.copy()
num_bridge_edges = 30
for _ in range(num_bridge_edges):
    # Chọn ngẫu nhiên 2 nút từ 2 cộng đồng khác nhau
    node1 = np.random.choice(list(ground_truth_communities[0]))
    node2 = np.random.choice(list(ground_truth_communities[1]))
    if not G2.has_edge(node1, node2):
        G2.add_edge(node1, node2)

# Tính lại Modularity ở T=2 trên cấu trúc của G2, nhưng vẫn dựa trên phân chia cộng đồng gốc
modularity_t2 = nx.community.modularity(G2, ground_truth_communities)
print(f"Modularity tại T=2 (cấu trúc mờ hơn):  {modularity_t2:.4f}")
print("-" * 50 + "\n")


# ==============================================================================
# BÀI TẬP 4: Đánh giá độ chính xác bằng dữ liệu LFR
# ==============================================================================
print("--- BÀI TẬP 4: Đánh giá độ chính xác của Louvain ---")

# Thử nghiệm với 2 mức độ "mờ" khác nhau của cộng đồng
mixing_params = [0.1, 0.6] # mu=0.1 (rõ), mu=0.6 (mờ)

for mu in mixing_params:
    # 1. Tạo đồ thị LFR benchmark
    # Đồ thị LFR được thiết kế đặc biệt để có cấu trúc cộng đồng biết trước
    G_lfr = nx.LFR_benchmark_graph(n=250, tau1=3, tau2=1.5, mu=mu, average_degree=5,
                                   min_community=20, seed=42)

    # Lấy ra nhãn cộng đồng gốc (ground truth) từ đồ thị
    ground_truth_comms_lfr = {frozenset(G_lfr.nodes[v]['community']) for v in G_lfr}
    ground_truth_labels = [0] * len(G_lfr.nodes)
    for i, comm in enumerate(ground_truth_comms_lfr):
        for node in comm:
            ground_truth_labels[node] = i

    # 2. Chạy thuật toán Louvain để tìm cộng đồng
    detected_partition = community_louvain.best_partition(G_lfr)
    detected_labels = list(detected_partition.values())

    # 3. Đánh giá độ chính xác bằng chỉ số Adjusted Rand Index (ARI)
    # ARI = 1.0 là trùng khớp hoàn hảo, ARI = 0.0 là đoán ngẫu nhiên
    ari_score = adjusted_rand_score(ground_truth_labels, detected_labels)
    
    print(f"Kết quả với tham số trộn (mu) = {mu}:")
    print(f"=> Độ chính xác của Louvain (ARI Score): {ari_score:.4f}")

print("-" * 50)

--- BÀI TẬP 2: So sánh các thuật toán ---
[Louvain]        => Đã tìm thấy 4 cộng đồng.
[Girvan-Newman]  => Đã tìm thấy 2 cộng đồng ở lần chia đầu tiên.
[Spectral]       => Đã tìm thấy 2 cộng đồng như đã yêu cầu.
--------------------------------------------------

--- BÀI TẬP 3: Mô phỏng cộng đồng biến đổi ---
Modularity tại T=1 (cấu trúc rõ rệt): 0.6074
Modularity tại T=2 (cấu trúc mờ hơn):  0.5669
--------------------------------------------------

--- BÀI TẬP 4: Đánh giá độ chính xác của Louvain ---
Kết quả với tham số trộn (mu) = 0.1:
=> Độ chính xác của Louvain (ARI Score): 0.9330
Kết quả với tham số trộn (mu) = 0.6:
=> Độ chính xác của Louvain (ARI Score): 0.0047
--------------------------------------------------
